In [4]:
import cv2
import numpy as np

In [5]:
def calculateSpatialInfo(img: np.ndarray) -> float:
    """
    Calculate spatial information from a grayscale image using Sobel filters.

    Args:
        img: A grayscale image as a numpy array.

    Returns:
        The spatial information as a float.
    """
    sh = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=1)
    sv = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=1)

    sobel_image = np.sqrt(np.square(sh) + np.square(sv))

    si_mean = np.sum(sobel_image) / (sobel_image.shape[0] * sobel_image.shape[1])
    si_rms = np.sqrt(np.sum(sobel_image ** 2) / (sobel_image.shape[0] * sobel_image.shape[1]))
    si_stdev = np.sqrt(np.sum(sobel_image ** 2 - si_mean ** 2) / (sobel_image.shape[0] * sobel_image.shape[1]))

    return si_stdev